In [1]:
import datetime
import calendar

from pycds import *
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import and_, or_
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pgpasslib

In [3]:
connection_string = "postgresql+psycopg2://nannau@dbmsc.pcic.uvic.ca/msc"
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()

obs = session.query(Obs).limit(50).all()

In [4]:
year = 2000
month = 2
day = 1
interval = 1
start_time = datetime.datetime(year, month, day)
end_time = datetime.datetime(year, month + interval, day)

In [29]:

def baseline(start_time, end_time, limit = 100):
    query = session.query(Obs.datum, Obs.time, Variable.standard_name, History.station_id).filter(Obs.time <= end_time).filter(Obs.time >= start_time).join(History).join(Variable).limit(limit)
    return query

def temperature_query(limit = 100):
    query = baseline(limit).filter()

In [31]:
pd.read_sql(baseline(start_time, end_time, 50000).statement, engine)

,datum,obs_time,standard_name,station_id
0,0.0,2000-02-01,thickness_of_snowfall_amount,122
1,0.0,2000-02-02,thickness_of_snowfall_amount,122
2,0.0,2000-02-03,thickness_of_snowfall_amount,122
3,0.0,2000-02-04,thickness_of_snowfall_amount,122
4,0.0,2000-02-05,thickness_of_snowfall_amount,122
5,0.0,2000-02-06,thickness_of_snowfall_amount,122
6,0.0,2000-02-07,thickness_of_snowfall_amount,122
7,0.0,2000-02-08,thickness_of_snowfall_amount,122
8,0.0,2000-02-09,thickness_of_snowfall_amount,122
9,0.0,2000-02-10,thickness_of_snowfall_amount,122
